In [3]:
import pandas as pd
import numpy as np

def get_possible_next_states(state, F, states_count):
    possible_next_states = []
    for i in range(states_count):
        if F[state, i] == 1: 
            possible_next_states.append(i)
    
    return possible_next_states

def get_random_next_state(state, F, states_count):
    possible_next_states = get_possible_next_states(state, F, states_count)
    next_state = possible_next_states[np.random.randint(0, len(possible_next_states))]
    
    return next_state

F = np.loadtxt("feasibility_matrix.csv", dtype="int", delimiter=',')

R = np.loadtxt("reward_matrix.csv", dtype="float", delimiter=',')

Q = np.zeros(shape=[15,15], dtype=np.float32)

display(pd.DataFrame(Q, dtype=float).style.format(precision=2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [4]:
def train(F, R, Q, gamma, lr, goal_state, states_count, episodes):
    for i in range(0, episodes):
        # Selects a random start state
        current_state = np.random.randint(0, states_count)

        # Continues till goal state is reached
        while(True):
            # Selects a random next state from the current state
            next_state = get_random_next_state(current_state, F, states_count)
            
            # Gets all possible states from that next state
            possible_next_next_states = get_possible_next_states(next_state, F, states_count)

            # Compares the Q value between two possible next states
            max_Q = -9999.99
            for j in range(len(possible_next_next_states)):
                next_next_state = possible_next_next_states[j]
                q = Q[next_state, next_next_state]
                if q > max_Q:
                    max_Q = q
            
            # Updates the Q value using Bellman equation [refer maze image caption]
            Q[current_state][next_state] = \
                ((1 - lr) * Q[current_state][next_state]) + (lr * (R[current_state][next_state] + (gamma * max_Q)))

            # Changes state by considering next state as current state and 
            # the training continues till goal state is reached
            current_state = next_state
            
            if current_state == goal_state:
                break
                
# Sets hyperparameters

gamma = 0.5        # discount factor
lr = 0.5           # learning_rate
goal_state = 14
states_count = 15
episodes = 1000

np.random.seed(42)

# starts training
train(F, R, Q, gamma, lr, goal_state, states_count, episodes)

# Prints Q matrix generated out of training
display(pd.DataFrame(Q, dtype=float).style.format(precision=2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,1.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.62,0.00,2.50,0.00,0.00,0.00,0.62,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,1.25,0.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00,0.00,0.00,0.00
5,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.62,0.00,0.00,0.00,0.16,0.00,0.00
8,0.00,0.00,0.00,1.25,0.00,0.00,0.00,0.31,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,2.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00


In [5]:
# The function walk() 

def print_shortest_path(start_state, goal_state, Q):
    current_state = start_state
    print(str(current_state) + "->", end="")
    
    # Loops till goal is reached and keeps on tracing the path
    while current_state != goal_state:
        # Chooses the best state from possible states and keep on printing
        next_state = np.argmax(Q[current_state])
        print(str(next_state) + "->", end="")
        
        # Considers next state as current state and continiues till goal is reached
        current_state = next_state
    
    print("Goal Reached.\n")
    
# Performs few tests for agent to get the shortest path

start_state = 8
print("Best path to reach goal from state {0} to goal state {1}".format(start_state, goal_state))
print_shortest_path(start_state, goal_state, Q)

start_state = 13
print("Best path to reach goal from state {0} to goal state {1}".format(start_state, goal_state))
print_shortest_path(start_state, goal_state, Q)

start_state = 6
print("Best path to reach goal from state {0} to goal state {1}".format(start_state, goal_state))
print_shortest_path(start_state, goal_state, Q)

start_state = 1
print("Best path to reach goal from state {0} to goal state {1}".format(start_state, goal_state))
print_shortest_path(start_state, goal_state, Q)

Best path to reach goal from state 8 to goal state 14
8->3->4->9->14->Goal Reached.

Best path to reach goal from state 13 to goal state 14
13->12->7->8->3->4->9->14->Goal Reached.

Best path to reach goal from state 6 to goal state 14
6->5->10->11->12->7->8->3->4->9->14->Goal Reached.

Best path to reach goal from state 1 to goal state 14
1->0->5->10->11->12->7->8->3->4->9->14->Goal Reached.

